In [1]:
#!/usr/bin/env python3

########################################################################
# File: Chan_Nicholas_randomizedMotifSearch.ipynb
# Purpose: to find the promoter motif
#   main(infile='FILE_PATH',outfile='FILE_PATH', inCL=['-i 1000', '-p 1', '-k 13'])
#
# Author: Nicholas Chan
# History: 10/10/2021 Created
########################################################################

# Command Line Class
Provided by Dr. B for parsing command line arguments

In [2]:
########################################################################
# CommandLine
########################################################################


class CommandLine():
    """
    Handle the command line, usage and help requests.

    CommandLine uses argparse,
    it implements a standard command line argument parser with various argument options,
    a standard usage and help, and an error termination exception Usage.
    """

    def __init__(self, inOpts = None):
        """
        CommandLine constructor.
        
        Implement a parser to interpret the command line argv string using argparse.
        """
        import argparse
        self.parser = argparse.ArgumentParser(
            description='Program prolog - a brief description of what this thing does',
            epilog='Program epilog - some other stuff you feel compelled to say',
            add_help=True,  # default is True
            prefix_chars='-',
            usage='%(prog)s [options] -option1[default] <input >output'
            )

        self.parser.add_argument('-i', '--iterations', type=int, default=1000, action='store', help='number of times of iterations to find consensus motif')
        self.parser.add_argument('-k', '--motifLength', type=int, default=13, action='store', help='length of the target promoter motif')
        self.parser.add_argument('-p', '--pseudocount', type=float, default=1, action='store', help='number of pseudocounts')
        # Command line option to use Gibbs sampling to find the optimal consensus motif.
#         self.parser.add_argument('-g', '--gibbsampling', type=float, default=1, action='store', help='implement Gibbs sampling')

        if inOpts is None:
            self.args = self.parser.parse_args()
        else:
            self.args = self.parser.parse_args(inOpts)

# FastAreader Class
Provided by Dr. B for reading Fasta Files.

In [3]:

import sys

class FastAreader():
    """
    Read in files and preprocess.
    """
    def __init__(self, fname=''):
        """ Contructor: saves attribute fname. """
        self.fname = fname

    def doOpen(self):
        """ Open a file."""
        if self.fname == '':
            return sys.stdin
        else:
            return open(self.fname)

    def readFasta(self):
        """ Read in a fasta file and yield header and sequences separately"""
        header = ''
        sequence = ''

        with self.doOpen() as fileH:

            header = ''
            sequence = ''

            # skip to first fasta header
            line = fileH.readline()
            while not line.startswith('>'):
                line = fileH.readline()
            header = line[1:].rstrip()

            # Separate headers and sequences
            for line in fileH:
                if line.startswith('>'):
                    yield header, sequence
                    header = line[1:].rstrip()
                    sequence = ''
                else:
                    sequence += ''.join(line.rstrip().split()).upper()

        yield header, sequence

# Assignment 1: CRISPR Promoter Sequences

For this assignment I used the Random Motif Search algorithm described by Dr. B and Chapter 2 of Binf. Algorithms. The goal of this 
assignment was to identify likely motifs in given sequences which could be DnaA binding sites. This algorithm first gathers information
on the base composition of all given sequences to create a NULL model to be compared to our experimental models. Our experimental models
first start out as a list of randomly selected motifs of size k. From there, an initial profile matrix is made (including pseudo counts
if any). We then grab all possible k-mer permutations in each sequence and apply our newly made profile matrix to develop a new set of 
motifs. With a new set of motifs, we are able to make a new profile matrix. We can compare the relative entropy scores between our initial 
model and our new model made from the previous profile matrix. If the Relatvie Entropy score of our new model is higher than the previous,
we continue making new models from previous models. Else we stop and return the consensus sequence and Relative Entropy score from the 
best performing model.

## Shannon's Entropy
$H(x) = -\sum_{i = 1}^{n} P(x_{i}) log_{2} (P(x_{i}))$ 
<br/>
$H(x) = -\sum_{i \in [A,C,G,T]} P(x_{i}) log_{2} (P(x_{i}))$
- An encoding cost

## Relative Entropy
$D_{KL}(P||Q) = \sum_{i \in [A,C,G,T]} P(x_{i}) log_{2} \frac{P(x_{i})} {Q(x_{i})}$
- A distance between 2 profile probability distributions
- P and Q are 2 probability profile distributions
- We look at the log of a ratio of 2 probabilities (a comparison)
- We want to compare our observed profile to just the base composition of all our sequences
- Model Q is our NULL model
    - This is the probability of each of our 4 bases
- Model P describes our experimental model
    - This is the probability of each of our 4 bases in our selected motifs

In [4]:
import sys
import os 
import numpy as np

# Handle iterations i in main function
class RandomMotifSearch():
    '''
    The RandomMotifSearch class finds the null model, possible k-mers, probability profile,
    likely motifs of size k, and relative entropy scores of profiles of a given set of sequences.
    All these can be returned individually with a call to a RandomMotifSearch object. An object 
    of RandomMotifSearch requires k: length of a motif to search for, p: number of pseudo counts,
    seqList: list of sequences from a fasta file (FastAreader ouput), and seqString: a string of all 
    the sequences from a fasta file (FastAreader ouput). 
    '''
    def __init__(self, k:'int', p:'int', seqList:'list', seqString:'str'):
        self.p = p # Pseudo counts
        self.k = k # k-mer length
        self.seqList = seqList # List of input sequences
        self.seqString = seqString # String of all input sequences
        
        self.nullModel = self.nullMod() # Null model generated from collection of all sequences
        self.kmerPermutations = self.generateKMerPerms() # A list of lists of k-mer permutations of size k from a sequence
        self.motifList = self.motifify() # First motif list, use reMotifify to generate a new lsit
        self.profileMatrix = self.profilify() # First motif profile matrix, use profilify again with a new motif list to generate a new profile
        self.REscore = self.bestProfile() # Best recorded RE Score from the current motif probability profile

    def nullMod(self) -> 'dict()':
        '''
        Returns null model with pseudocounts.
        '''
        nullProfDict = dict()
        countA = self.seqString.count('A')
        countC = self.seqString.count('C')
        countG = self.seqString.count('G')
        countT = self.seqString.count('T')
        countTot = countA + countC + countG + countT
        nullProfDict['A'] = (countA + self.p)/(countTot + self.p*4)
        nullProfDict['C'] = (countC + self.p)/(countTot + self.p*4)
        nullProfDict['G'] = (countG + self.p)/(countTot + self.p*4)
        nullProfDict['T'] = (countT + self.p)/(countTot + self.p*4)
        return nullProfDict
    
    def motifify(self) -> 'list': # Randomly generates motifs, independent
        ''' 
        Params: (self.seqList: 'list', self.k: 'int of motif length')
        Generates n number of sequences of length k (list of k-mers/motifs). 
        In other worrds, makes a list of randomly selected 
        k-mers/motifs from given list of seqs.
        '''
        motifs = []
        for seq in self.seqList: # Can be read as for each sequence in the given fasta file
            indexRange = np.random.randint(0,len(seq)-self.k) # Sets a valid range for randomly pulling k-mers from a sequence 
            motifs.append(seq[indexRange:indexRange + self.k]) # Randomly selects a motif from a sequence and appends it to a list
        return motifs # Returns a list of motifs, 1 motif per sequence in fasta file
    
    # Counts check out, all columns add up to 1 and fractions match decimals
    def profilify(self) -> 'dict(lists)': # Independent
        ''' 
        Params: (motifList: 'list', p:'int'=0)
        Generates a profile matrix given a list of k-mers/motifs.
        Pseudocounts are added into the Experimental and NULL models
        in this method.
        '''
        # A-C-G-T correspond to 0-1-2-3
        matRowCount = len(self.motifList)
        motifLength = len(self.motifList[0])
        # Dictionary matrix where rows are bases [A,C,T,G] and cols are motif indices
        profileMatrix = dict(A=np.array([0]*motifLength), C=np.array([0]*motifLength), T=np.array([0]*motifLength), G=np.array([0]*motifLength))
        for motif in self.motifList: # Iterate first over motifs in motif list (input)
            idx=0 # Keep index of motif positions
            for base in motif: # Iterate over motif bases
                profileMatrix[base][idx]+=1
                idx+=1
        if self.p > 0: # If there are pseudocounts to be added
            for key in profileMatrix.keys():
                profileMatrix[key] = np.divide(profileMatrix[key] + self.p, matRowCount + (self.p*4)) # divides a list of motif base frequencies + pseudocount p by total bases per index + pseudocounts p*4
            return profileMatrix
        # If pseudocounts are 0, might not need em
        else: # If there are no pseudocounts to be added
            for key in profileMatrix.keys():
                profileMatrix[key] = np.divide(profileMatrix[key],matRowCount) # divides a list of motif base frequencies by total bases per index
            return profileMatrix  
        
    def findProb(self, kmer:'str') -> 'float':
        ''' 
        Params: (profileMatrix:'dict(lists)',kmer:'str')
        Finds the probability of a whole k-mer given a Profile.
        Find individual probabilities of each base in motif and finds product from each position.
        '''
        totProb = 1
        for idx in range(len(kmer)): # Counts through each base position in the k-mer
            totProb *= self.profileMatrix[kmer[idx]][idx] 
        return totProb # Returns the product of all base probabilities in a k-mer (a k-mer's likelihood)
    
    def generateKMerPerms(self) -> 'list[lists]': 
        ''' 
        Params: (seqList: 'List of raw sequences', k: 'int of k-mer length')
        Generates a list of lists of all k-mer permutations from raw sequences 
        '''
        kmerMat = []
        for seq in self.seqList: # Goes through each sequence from the input
            kmerList = []
            for idx in range(len(seq)-self.k): # Uses sliding frame method to collect all possible k-mers in a sequence
                kmerList.append(seq[idx:idx+self.k])
            kmerMat.append(kmerList)
        return kmerMat # Returns a list of all possible k-mers per sequence
    
    def reMotifify(self) -> 'list of k-mers with the maximum probs':
        ''''
        Params: (kmerPermutations:'list of lists, profileMatrix:'4 x k dict of probabilities')
        Same type of output as motifify.
        Use this to generate motifs with maximum likelihood after initial random motif generation.
        A PERMUTATION MATRIX IS READ IN (list of lists)
        Each row contains all possible motif permutations per given sequence.
        '''
        maxPermList = []
        for permList in self.kmerPermutations:
            probList = []
            for perm in permList:
                probList.append(self.findProb(perm)) # Calculates probability of a motif given a profile and appends the probability
            maxPermList.append(permList[probList.index(max(probList))]) # Finds index of the motif with the highest probability in a PERMUTATION LIST and appends the permutation to maxPermList
        return maxPermList

    def findConsensus(self): # Can be called at end AFTER calling rms.bestProfile()
        '''
        Params: (profMat:'dict of np arrays')
        Finds a consensus sequence from the current profile matrix.
        '''
        motifLength = len(self.profileMatrix['A'])
        baseList = ['A','C','T','G']
        conSeq = ''
        for idx in range(motifLength):
            baseColList = []
            for base in baseList:
                baseColList.append((base,self.profileMatrix[base][idx]))
            baseColList.sort(key=lambda x:x[1])
            conSeq += baseColList[::-1][0][0] # Way of getting to value as baseColList looks like [['A',1],['B',2], ...,['F',6]]
        return conSeq
    
    def relativeEntropy(self, expModel:'dict(lists)'):
        '''
        Exp model is the profile matrix you want to score.
        This method calculates the relative entropy score of a profile matrix.
        '''
        colLen = len(expModel['A'])
        bases = ['A', 'T', 'C', 'G']
        fullPart = []# list for sum of cols of RE eq
        for col in range(colLen): # calculation for RE for 1 column
            basePart = [] # list for inner sum of RE eq
            for base in bases: # calculation for RE for 1 base
                frac = expModel[base][col]/self.nullModel[base]
                logged = np.log2(frac)
                bpart = expModel[base][col]*logged
                basePart.append(bpart)
            fullPart.append(sum(basePart))
        return sum(fullPart)       
    
    def bestProfile(self):
        '''
        This method iteratively improves the approximation of our 
        profile matrix and RE Score. Profile matrices are iteratively
        made form preexisting ones until RE Score no longer improves.
        '''
        RE = self.relativeEntropy(self.profileMatrix) # Saves previous RE Score
        diff = 0 # Saves a value for difference between Prev and Curr RE Scores
        First = True # Special condition for First
        while (diff > 0.0 or First):
            self.motifList = self.reMotifify() # Builds a new motif list with the existing profile matrix
            self.profileMatrix = self.profilify() # Builds a new profile matrix to replace the existing one
            First = False # Method is no longer in First state
            diff = self.relativeEntropy(self.profileMatrix) - RE # Creates a new difference between the previous and new existing RE Scores
            RE = self.relativeEntropy(self.profileMatrix) # Sets the existing RE Score to the newest one
        return RE # Returns RE Score when no further change occurs


# Main function 
Main function is written here. This function handles argument parsing, input, and output.

In [5]:
def main(infile, outfile='', inCL=None):
    '''
    This is the main function. Arguments are parsed here with the CommandLine class
    provided by Dr. B. Infile and outfile are also read here, with an outfile file being 
    optional. IF no outfile is provided, output is printed. Infile is read in through the
    readData function which returns fasta file information for the RandomMotifSearch to read.
    This way, input only needs to be read once. RandomMotifSearch is called i number of times,
    with the relative entropy score and consensus sequence being stored in the motifAndScore list.
    The highest RE Score and its corresponding consensus sequence are written to outup.
    '''
    myCommandLine = CommandLine(inCL)
    i = myCommandLine.args.iterations
    k = myCommandLine.args.motifLength
    p = myCommandLine.args.pseudocount
    
    def readData(infile:'str') -> 'tuple(list,str)':
        '''
        Reads in fasta file as input. 
        Returns [0]: a list of seqs [1]: a string of catted seqs.
        '''
        seqList = []
        readData = FastAreader(infile).readFasta()
        for line in readData:
            seqList.append(line[1]) # Append seq and not head to seqList
        seqString = ''.join(seqList) # After all seq lines have been added to seqList, cat all into 1 str
        return (seqList,seqString)
    
    data = readData(infile) # Saves output of FastAreader(infile).readFasta() as (seqList,seqString)
    seqList = data[0] # Saves a list of sequences
    seqString = data[1] # Saves a string of sequences
    
    motifAndScore = [] # Collects consensus sequences and RE Scores for each run of RandomMotifSearch()
    for idx in range(i):
        myRandomMotifSearch = RandomMotifSearch(k, p, seqList,seqString) # Each call of RandomMotifSearch() starts with a non-deterministic random motif selection step
        motifAndScore.append([myRandomMotifSearch.bestProfile(), myRandomMotifSearch.findConsensus()]) 
    motifAndScore.sort(key=lambda x:x[0]) # Sort list in ascending order by RE Score
    
    if len(outfile) > 0:
        with open(outfile, 'w') as myfile:
            myfile.write(f"{motifAndScore[-1][1]}\t{motifAndScore[-1][0]}")
    else:
        print(f"{motifAndScore[-1][1]}\t{motifAndScore[-1][0]}")

In [6]:
if __name__ == "__main__":
    '''
    This is where the main function is executed.
    Usage:
        main(infile='FILE_PATH', outfile='FILE_PATH', inCL=[-ipk] )
    Arguments:
        -i
            Number of RandomMotifSearch iterations
        -p
            Number of pseudo counts
        -k
            Length of motif to search for
    '''
    main(infile='Data/pcaCrisprs',outfile='', inCL=['-i 1000', '-p 1', '-k 13'])

CAAAGAAAAACTT	7.783555496977213


In [7]:
# INSPECTION

# INSPECTION TEAM
# Jodi Lee
# Maxim Firsov
# Hsiang-Yun Lu (Eloise)
# Gabriel Aguiar

# RESPONSES
# - Markdown comments
# - Get more docstrings
# - Make change so that infile is only read once
# - Finish main function

# CORRECTIONS
# - Made markdown comments
# - Made more docstrings and comments
# - Infile is only read once in the main function
# - Main function was put together